In [1]:
from models.DnCNN.DnCNN import create_DnCNN
from models.DnCNN.DnCNNwithRL import create_DnCNNwithRL
from models.NAFNet.NAFNet import create_NAFNet
from models.RDN.RDN import create_RDN
from models.CGNet.CGNet import create_CGNet
from utils.datasets import ImageDataset
from torch.utils.data import DataLoader, random_split
from utils.training import train, denoise_image, PSNR, CUDAstatistics
import torch
import numpy as np
import matplotlib.pyplot as plt
import os

model = create_DnCNNwithRL().cuda() 
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = None
criterion = torch.nn.MSELoss()
num_epochs = 10
batch_size = 4
display_images_per_epoch = 5

image_dir = 'C:/Users/jugou/superresolution_images4'  # Change this to your image folder path

val_size = 0.1   # Percentage of the dataset that will be used for validation
dataset = ImageDataset(image_dir)
train_size = int((1 - val_size) * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


CUDAstatistics()
train_losses, val_losses, train_psnrs, val_psnrs = train(model, train_loader, val_loader, optimizer, scheduler, criterion, num_epochs, display_images_per_epoch, "DnCNNwithRLSuperResol")




# Example of denoising an image
noisy_image, clean_image = train_dataset[1]
denoised_image = denoise_image(model, noisy_image)
noisy_image_np = noisy_image.numpy().transpose(1, 2, 0) * 255.0
clean_image_np = clean_image.numpy().transpose(1, 2, 0) * 255.0
denoised_image_np = denoised_image.transpose(1, 2, 0) * 255.0
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.imshow(clean_image_np.astype(np.uint8))
plt.title('Original Image')
plt.subplot(1, 3, 2)
plt.imshow(noisy_image_np.astype(np.uint8))
plt.title('Noisy Image')
plt.subplot(1, 3, 3)
plt.imshow(denoised_image_np.astype(np.uint8))
plt.title('Denoised Image')
plt.show()
# Calculate PSNR
psnr_noisy, psnr_denoised = PSNR(clean_image, noisy_image, torch.tensor(denoised_image))
print(f"PSNR Noisy: {psnr_noisy}, PSNR Denoised: {psnr_denoised}")
# Calculate the pixel with the most difference
diff = np.abs(clean_image_np - denoised_image_np)
diff_max = np.max(diff)
diff_pos = np.unravel_index(np.argmax(diff, axis=None), diff.shape)
print(f"Maximum difference: {diff_max}")
print(f"Position of maximum difference: {diff_pos}")
print(f"Value in original image: {clean_image_np[diff_pos]}")
print(f"Value in denoised image: {denoised_image_np[diff_pos]}")


KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), 'DnCNNwithRLsuperresolution4only23PSNR.pth')